# Zona de Importaciones

In [ ]:
import pandas as pd
import numpy as np
import random
import shutil
import tempfile
import yt_dlp
#from pytube import YouTube
import speech_recognition as sr
import whisper
from pydub import AudioSegment
import dask
from dask import delayed
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
import time
# import concurrent.futures
import os
import re

from vosk import Model, KaldiRecognizer
# import deepspeech
import wave
import json

import subprocess

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa

import torchaudio
# from espnet_model_zoo.downloader import ModelDownloader

import re
import os

# os.environ["KERAS_BACKEND"] = "tensorflow"

# from glob import glob
import tensorflow as tf
import keras
from keras import layers

# pattern_wav_name = re.compile(r'([^/\\\.]+)')


# Variables de entorno

In [ ]:
COOKIES_FILE_PATH = '../conf/youtube.com_cookies.txt'
INPUT_FILE_PATH = '../data/01_raw/canciones_urls.xlsx'
OUTPUT_PATH = '../data/01_raw/'
AUDIO_PATH = '../data/03_primary/'


# Lista de User-Agents para rotar
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
]

# Cargar Dataframe

In [ ]:
df = pd.read_excel(INPUT_FILE_PATH)

In [ ]:
df.drop_duplicates(inplace=True)

# Descargar Audio (.wav)

In [ ]:
def descargar_audio(url, audio_name):
    """
    Descarga un audio desde una URL usando yt-dlp.
    
    Args:
        url: URL del video a descargar
        audio_name: Nombre base del archivo de audio (sin extensión)
        
    Returns:
        bool: True si la descarga fue exitosa, False en caso contrario
    """
    temp_cookies = None
    try:
        # Validar y preparar la ruta de salida
        audio_path = os.path.join(AUDIO_PATH, audio_name)
        if not os.path.exists(AUDIO_PATH):
            os.makedirs(AUDIO_PATH)
        
        # Crear archivo temporal de cookies
        temp_cookies = tempfile.NamedTemporaryFile(delete=False)
        shutil.copy(COOKIES_FILE_PATH, temp_cookies.name)
        
        # Configurar opciones de yt-dlp
        ydl_opts = {
            'quiet': True,
            'ignoreerrors': True,
            'format': 'bestaudio/best', 
            'outtmpl': audio_path + '.%(ext)s',
            'extractaudio': True,
            'audioformat': 'wav',
            'cookiefile': temp_cookies.name,
            'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.36',
            'retries': 10,
            'fragment_retries': 10,
            'extractor_args': {
                'youtube': {
                    'skip': ['dash', 'hls']
                }
            },
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            }],
            'prefer_ffmpeg': True,
            'verbose': True
        }
        
        # Descargar el audio
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
            
        # Verificar que el archivo se creó
        for ext in ['.mp3', '.webm', '.m4a']:
            output_file = f"{audio_path}{ext}"
            if os.path.exists(output_file):
                return True
            
        print(f"Error: No se pudo encontrar el archivo descargado {audio_name}")
        return False
        
    except yt_dlp.DownloadError as e:
        print(f'Error al descargar el audio: {e}')
        return False
    except IOError as e:
        print(f'Error de E/S con los archivos: {e}')
        return False
    except Exception as e:
        print(f'Error inesperado: {e}')
        return False
    finally:
        # Limpiar archivo temporal de cookies
        if temp_cookies:
            try:
                temp_cookies.close()
                os.unlink(temp_cookies.name)
            except Exception as e:
                print(f"Error al limpiar archivos temporales: {e}")

In [ ]:
# Función para descargar un audio usando yt-dlp
def descargar_audio_bash(url, nombre_audio):
    try:
        comando = [
            "yt-dlp",
            "-x",  # Extraer audio
            "--audio-format", "wav",  # Formato WAV
            "--output", f"{AUDIO_PATH}/{nombre_audio}.%(ext)s",  # Ruta de salida
            url
        ]
        # Ejecutar comando en terminal
        subprocess.run(comando, check=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error al descargar {nombre_audio}: {e}")
        return False

In [ ]:
def get_audio(url, audio_name):
    time.sleep(random.randint(5, 10))
    desc = descargar_audio_bash(url, audio_name)
    return desc

# Paralelización en la descarga

In [ ]:
tasks = [delayed(get_audio)(url, audio_name) for url, audio_name in df[['URL','AUDIO_NAME']].values]

In [ ]:
with ProgressBar():
    df['AUDIO_DESCARGADO'] = dask.compute(*tasks, scheduler='threads')

In [ ]:
df['AUDIO_DESCARGADO'].value_counts()